In [9]:
from scipy.optimize import minimize
import numpy as np

In [239]:
def get_nu_star(xi, params):

    def objective(nu):

        if xi(nu, **params)>0:
            return -(np.log(xi(nu, **params)) - 0.5*np.dot(nu.T, nu))
        else:
            return 1e80
        
    nu0 = np.zeros(params["d"])

    result = minimize(objective, nu0, method='SLSQP')

    if result.success:
        return result.x
    else:
        raise ValueError("L'optimisation n'a pas réussi à converger.")

In [236]:
r = 0.05
T = 1
s0 = 50
sigma = 0.1
K = 50

B = 60

In [252]:
def asian_function(y, S0, v, mu, maturity, strike, rate, d):

    k = np.arange(1, d + 1)
    y_cum = np.cumsum(y)
    payoff = np.exp(-rate*maturity)*max(S0*np.sum(np.exp(mu*(k*T)/d + v*np.sqrt(T/d)*y_cum))/d - strike, 0)

    return payoff

In [253]:
d = 16
T = 1
r = 0.05

S0 = 50
K = 45
v = 0.1

params_asian = {
    "d": d,
    "S0": S0,
    "v": v,
    "mu": r-0.5*v**2,
    "maturity": T,
    "strike": K,
    "rate": r
}

In [254]:
nu = get_nu_star(asian_function, params_asian)/0.42
nu

array([0.41409413, 0.38929433, 0.36432277, 0.33918466, 0.31388597,
       0.28843238, 0.26282949, 0.23708368, 0.21120092, 0.18518755,
       0.15905014, 0.13279537, 0.10642987, 0.07996067, 0.05339449,
       0.02673845])

In [267]:
def knock_out_function(y, S0, v, mu, maturity, strike, rate, barrier, d):

    d = len(y)
    k = np.arange(1, d + 1)
    y_cum = np.cumsum(y)

    s = S0*np.exp(mu*maturity + v*np.sqrt(maturity/d)*y_cum[-1])
    payoff = np.exp(-rate*maturity)*max(S0*np.sum(np.exp(mu*(k*maturity)/d + v*np.sqrt(maturity/d)*y_cum))/d - strike, 0)*(1 if s <= barrier else 0)

    return payoff

In [270]:
d = 16
T = 1
r = 0.05

S0 = 50
K = 50
B = 60
v = 0.1

params_barrier = {
    "d": d,
    "S0": S0,
    "v": v,
    "mu": r-0.5*v**2,
    "maturity": T,
    "strike": K,
    "rate": r, 
    "barrier": B
}

In [272]:
nu = get_nu_star(knock_out_function, params_barrier)/0.84
nu

array([0.40948557, 0.38548036, 0.36121145, 0.33668891, 0.31192197,
       0.2869225 , 0.26170077, 0.23626848, 0.21063748, 0.1848205 ,
       0.15882961, 0.13267859, 0.10638069, 0.07994989, 0.05339963,
       0.0267447 ])

In [273]:
def basket_options_function(y, S0, v, mu, sq_correl, maturity, strike, alpha, rate, d):

    y_tilde = np.dot(sq_correl, y)
    payoff = np.exp(-rate*maturity)*max(np.sum(alpha*S0*np.exp(mu*T + v*np.sqrt(maturity)*y_tilde)) - strike, 0)

    return payoff

In [274]:
d = 40
T = 1
r = 0.05

S0 = np.random.uniform(20, 80, d)
K = 45
alpha = np.ones(d)/d

c = 0.1
sigma = c*np.ones((d, d)) + (1-c)*np.eye(d)
v = np.linspace(0.1, 0.4, d)

params_basket = {
    "d": d,
    "S0": S0,
    "v": v,
    "mu": r-0.5*v**2,
    "sq_correl": np.linalg.cholesky(sigma),
    "maturity": T,
    "strike": K,
    "alpha": alpha,
    "rate": r
}

In [275]:
nu = get_nu_star(basket_options_function, params_basket)/0.41
nu

array([0.51348833, 0.44394845, 0.43429594, 0.35139928, 0.38121441,
       0.30647044, 0.34857792, 0.30098923, 0.29458778, 0.27000911,
       0.22611321, 0.22334599, 0.23443787, 0.25623879, 0.21426465,
       0.17079791, 0.17711434, 0.20669697, 0.16863798, 0.20787631,
       0.21194631, 0.24810199, 0.19194896, 0.16402126, 0.12493332,
       0.27173567, 0.25517783, 0.10795869, 0.16161881, 0.18370867,
       0.15716313, 0.12740188, 0.11707627, 0.19528741, 0.26184598,
       0.27994829, 0.23961907, 0.08303473, 0.16673978, 0.20455805])